<a href="https://colab.research.google.com/github/lsh950919/Portfolio/blob/main/Post_COVID_19_Project_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = False)

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/Colab Notebooks

/content/drive/My Drive/Colab Notebooks


In [ ]:
import pandas as pd

# 데이터 불러오기

In [ ]:
population = pd.read_csv('Project 2 data/fpopl.csv')

In [ ]:
dong_code = pd.read_csv('Project 2 data/adstrd_master.csv')

In [ ]:
card = pd.read_csv('Project 2 data/card_20200717.csv')

# 데이터 전처리

## 유동인구

In [ ]:
population.head()

,base_ymd,tmzon_se_code,sexdstn_se_code,agrde_se_code,adstrd_code,popltn_cascnt
0,20200101,0,M,age_00,11650560,15
1,20200101,0,M,age_00,11590620,6
2,20200101,0,M,age_00,11560710,4
3,20200101,0,M,age_00,11470680,12
4,20200101,0,M,age_00,11350665,6


In [ ]:
# 일별 유동인구로 변환 후 날짜 지정
population2 = population.groupby(['base_ymd','sexdstn_se_code','adstrd_code'])['popltn_cascnt'].sum().reset_index()
population2 = population2[(population2['base_ymd'] >= 20200104) & (population2['base_ymd'] <= 20200614)]

In [ ]:
population2.tail()

,base_ymd,sexdstn_se_code,adstrd_code,popltn_cascnt
141033,20200614,M,11740650,54512
141034,20200614,M,11740660,88335
141035,20200614,M,11740685,67273
141036,20200614,M,11740690,25537
141037,20200614,M,11740700,61868


## 행정동 코드

In [ ]:
dong_code.head()

,adstrd_code,adstrd_nm,brtc_nm,signgu_nm
0,11110515.0,청운효자동,서울특별시,종로구
1,11110530.0,사직동,서울특별시,종로구
2,11110540.0,삼청동,서울특별시,종로구
3,11110550.0,부암동,서울특별시,종로구
4,11110560.0,평창동,서울특별시,종로구


In [ ]:
# 행정동 코드 정수로 변환
dong_code['adstrd_code'] = dong_code['adstrd_code'].astype(int)

In [ ]:
dong_code.head()

,adstrd_code,adstrd_nm,brtc_nm,signgu_nm
0,11110515,청운효자동,서울특별시,종로구
1,11110530,사직동,서울특별시,종로구
2,11110540,삼청동,서울특별시,종로구
3,11110550,부암동,서울특별시,종로구
4,11110560,평창동,서울특별시,종로구


In [ ]:
# 서울 행정동 코드 추출
region_seoul = dong_code[dong_code['brtc_nm'].str.contains('서울')]
print(region_seoul.brtc_nm.unique())
region_seoul.info()
region_seoul.head()

['서울특별시']
<class 'pandas.core.frame.DataFrame'>
Int64Index: 425 entries, 0 to 424
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   adstrd_code  425 non-null    int64 
 1   adstrd_nm    425 non-null    object
 2   brtc_nm      425 non-null    object
 3   signgu_nm    425 non-null    object
dtypes: int64(1), object(3)
memory usage: 16.6+ KB


,adstrd_code,adstrd_nm,brtc_nm,signgu_nm
0,11110515,청운효자동,서울특별시,종로구
1,11110530,사직동,서울특별시,종로구
2,11110540,삼청동,서울특별시,종로구
3,11110550,부암동,서울특별시,종로구
4,11110560,평창동,서울특별시,종로구


## 유동인구와 행정동 정보 결합

In [ ]:
# 유동인구와 행정동정보 merge로 결합
region_pop = population2.merge(region_seoul, how = 'left')

In [ ]:
region_pop.head()

,base_ymd,sexdstn_se_code,adstrd_code,popltn_cascnt,adstrd_nm,brtc_nm,signgu_nm
0,20200104,F,11110515,32142,청운효자동,서울특별시,종로구
1,20200104,F,11110530,81625,사직동,서울특별시,종로구
2,20200104,F,11110540,18660,삼청동,서울특별시,종로구
3,20200104,F,11110550,37372,부암동,서울특별시,종로구
4,20200104,F,11110560,42261,평창동,서울특별시,종로구


### 각 성별 유동인구 합치기

In [ ]:
# 기본틀
region_pop2 = region_pop[region_pop['sexdstn_se_code'] == 'F'].copy()
region_pop2.drop('sexdstn_se_code', axis = 1, inplace = True)
region_pop2 = region_pop2.sort_values(by = ['base_ymd', 'adstrd_code']) # 후에 카드 데이터와 결합을 맞춰주기 위해
region_pop2

In [ ]:
region_pop2.head()

,base_ymd,adstrd_code,popltn_cascnt,adstrd_nm,brtc_nm,signgu_nm
0,20200104,11110515,32142,청운효자동,서울특별시,종로구
1,20200104,11110530,81625,사직동,서울특별시,종로구
2,20200104,11110540,18660,삼청동,서울특별시,종로구
3,20200104,11110550,37372,부암동,서울특별시,종로구
4,20200104,11110560,42261,평창동,서울특별시,종로구


In [ ]:
# 유동인구 통합 값 계산
pop_sum = region_pop.groupby(['base_ymd', 'adstrd_code', 'adstrd_nm', 'brtc_nm', 'signgu_nm']).sum().sort_values(
    by = ['base_ymd', 'adstrd_code']).reset_index().copy()
pop_sum = pop_sum.rename(columns = {'popltn_cascnt': 'popltn_cascnt2'}) 
pop_sum

,base_ymd,adstrd_code,adstrd_nm,brtc_nm,signgu_nm,popltn_cascnt2
0,20200104,11110515,청운효자동,서울특별시,종로구,73084
1,20200104,11110530,사직동,서울특별시,종로구,178588
2,20200104,11110540,삼청동,서울특별시,종로구,39915
3,20200104,11110550,부암동,서울특별시,종로구,90008
4,20200104,11110560,평창동,서울특별시,종로구,104662
...,...,...,...,...,...,...
69242,20200614,11740650,성내2동,서울특별시,강동구,105268
69243,20200614,11740660,성내3동,서울특별시,강동구,162537
69244,20200614,11740685,길동,서울특별시,강동구,127426
69245,20200614,11740690,둔촌1동,서울특별시,강동구,44733


In [ ]:
# 기본틀과 유동인구총합 결합
pop_sum_final = region_pop2.merge(pop_sum)
pop_sum_final

,base_ymd,adstrd_code,popltn_cascnt,adstrd_nm,brtc_nm,signgu_nm,popltn_cascnt2
0,20200104,11110515,32142,청운효자동,서울특별시,종로구,73084
1,20200104,11110530,81625,사직동,서울특별시,종로구,178588
2,20200104,11110540,18660,삼청동,서울특별시,종로구,39915
3,20200104,11110550,37372,부암동,서울특별시,종로구,90008
4,20200104,11110560,42261,평창동,서울특별시,종로구,104662
...,...,...,...,...,...,...,...
69242,20200614,11740650,50756,성내2동,서울특별시,강동구,105268
69243,20200614,11740660,74202,성내3동,서울특별시,강동구,162537
69244,20200614,11740685,60153,길동,서울특별시,강동구,127426
69245,20200614,11740690,19196,둔촌1동,서울특별시,강동구,44733


In [ ]:
# 유동인구 총합만 유지하고 성별 유동인구는 제거
pop_sum_final.drop('popltn_cascnt', axis = 1, inplace = True)

In [ ]:
pop_sum_final.head()

,base_ymd,adstrd_code,adstrd_nm,brtc_nm,signgu_nm,popltn_cascnt2
0,20200104,11110515,청운효자동,서울특별시,종로구,73084
1,20200104,11110530,사직동,서울특별시,종로구,178588
2,20200104,11110540,삼청동,서울특별시,종로구,39915
3,20200104,11110550,부암동,서울특별시,종로구,90008
4,20200104,11110560,평창동,서울특별시,종로구,104662


In [ ]:
# 구별 일별 유동인구
pop_sum_final2 = pop_sum_final.groupby(['base_ymd', 'brtc_nm', 'signgu_nm'])['popltn_cascnt2'].sum().reset_index().copy()
pop_sum_final2.head()

,base_ymd,brtc_nm,signgu_nm,popltn_cascnt2
0,20200104,서울특별시,강남구,4268654
1,20200104,서울특별시,강동구,2425402
2,20200104,서울특별시,강북구,1036862
3,20200104,서울특별시,강서구,2465692
4,20200104,서울특별시,관악구,1977717


In [ ]:
pop_sum_final2[pop_sum_final2['signgu_nm'] == '종로구'].head(1)

,base_ymd,brtc_nm,signgu_nm,popltn_cascnt2
22,20200104,서울특별시,종로구,1685580


In [ ]:
# 서울시 데이터만 다루니, brtc_nm 제거
pop_sum_final2.drop('brtc_nm', axis = 1, inplace = True)
# 열 이름 변경
pop_sum_final2.columns = ['날짜', '시군구', '유동인구']

In [ ]:
# 최종 유동인구 데이터
pop_sum_final2

,날짜,시군구,유동인구
0,20200104,강남구,4268654
1,20200104,강동구,2425402
2,20200104,강북구,1036862
3,20200104,강서구,2465692
4,20200104,관악구,1977717
...,...,...,...
4070,20200614,용산구,1848525
4071,20200614,은평구,1436435
4072,20200614,종로구,1269669
4073,20200614,중구,1166980


In [ ]:
pop_sum_final2.to_csv('pop_sum.csv', index = False)

## 카드데이터

In [ ]:
card.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3713952 entries, 0 to 3713951
Data columns (total 7 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   receipt_dttm          int64 
 1   adstrd_code           int64 
 2   adstrd_nm             object
 3   mrhst_induty_cl_code  int64 
 4   mrhst_induty_cl_nm    object
 5   selng_cascnt          int64 
 6   salamt                int64 
dtypes: int64(5), object(2)
memory usage: 198.3+ MB


In [ ]:
# 행정동 코드 100으로 나눈 후 정수로 변환
card.adstrd_code = (card.adstrd_code / 100).astype(int)
# 업종명의 여백 제거
card.mrhst_induty_cl_nm = card.mrhst_induty_cl_nm.str.replace(' ', '')

In [ ]:
# 유동인구 데이터와 열 이름 맞추기
card = card.rename(columns = {'receipt_dttm': 'base_ymd'})

In [ ]:
# 업종명 여백 제거 확인
card.mrhst_induty_cl_nm.unique()[0:50]

array(['약국', '치과의원', '한의원', '의원', '자동차정비', '기타교육', '보습학원', '일반서적', '정장',
       '슈퍼마켓', '편의점', '기타레져업', '노래방', '당구장', '헬스크럽', '스포츠레져용품', '기계공구',
       '기타건축자재', '기타음료식품', '농축수산품', '정육점', '제과점', '스넥', '주점', '서양음식',
       '중국음식', '일식회집', '일반한식', '화장품', '미용원', '주차장', '화원', '비영리',
       '레져업소(회원제형태)', '예체능학원', '문구용품', '사진관', '단란주점', '악세사리', '기타숙박업',
       '세탁소', '유흥주점', '안경', '유아원', '기타잡화', '인터넷종합Mall', '대형할인점', '애완동물',
       '학원(회원제형태)', '사무서비스'], dtype=object)

In [ ]:
card.head()

,base_ymd,adstrd_code,adstrd_nm,mrhst_induty_cl_code,mrhst_induty_cl_nm,selng_cascnt,salamt
0,20200104,11740660,성내3동,7041,약국,463,5843230
1,20200104,11740660,성내3동,7022,치과의원,33,7835550
2,20200104,11740660,성내3동,7021,한의원,53,4589800
3,20200104,11740660,성내3동,7020,의원,339,9267240
4,20200104,11740660,성내3동,6110,자동차정비,19,4441000


In [ ]:
region_seoul.head()

,adstrd_code,adstrd_nm,brtc_nm,signgu_nm
0,11110515,청운효자동,서울특별시,종로구
1,11110530,사직동,서울특별시,종로구
2,11110540,삼청동,서울특별시,종로구
3,11110550,부암동,서울특별시,종로구
4,11110560,평창동,서울특별시,종로구


### 구별로 수합

In [ ]:
# 카드데이터에 시도명과 시군구명 추가
card_dong = card.merge(region_seoul, how = 'left')
# 시도명 제거
card_dong.drop('brtc_nm', axis = 1, inplace = True)

In [ ]:
card_dong.sort_values(by = ['base_ymd', 'signgu_nm', 'mrhst_induty_cl_code']).head()

,base_ymd,adstrd_code,adstrd_nm,mrhst_induty_cl_code,mrhst_induty_cl_nm,selng_cascnt,salamt,signgu_nm
1334,20200104,11680580,삼성1동,1001,특급호텔,549,146315331,강남구
1066,20200104,11680650,역삼2동,1020,기타숙박업,86,16163585,강남구
1147,20200104,11680640,역삼1동,1020,기타숙박업,157,12910332,강남구
1297,20200104,11680590,삼성2동,1020,기타숙박업,49,2898484,강남구
1333,20200104,11680580,삼성1동,1020,기타숙박업,36,3054336,강남구


In [ ]:
# 일/동/업종명별 수치들을 일/구/업종명별로 수합
card_gu = card_dong.groupby(['base_ymd', 'mrhst_induty_cl_code', 'mrhst_induty_cl_nm', 'signgu_nm'])[['selng_cascnt', 'salamt']].sum().reset_index().copy()

In [ ]:
# 열 이름 변경
card_gu.columns = ['날짜', '업종코드', '업종명', '시군구', '판매건수', '매출액']

In [ ]:
card_gu.drop('업종코드', axis = 1, inplace = True)

In [ ]:
card_gu

,날짜,업종명,시군구,판매건수,매출액
0,20200104,특급호텔,강남구,549,146315331
1,20200104,특급호텔,중구,928,239811377
2,20200104,1급호텔,중구,38,6941145
3,20200104,기타숙박업,강남구,444,57200876
4,20200104,기타숙박업,강동구,180,7928000
...,...,...,...,...,...
573901,20200614,기타전문점,용산구,185,2793400
573902,20200614,기타전문점,은평구,42,1630800
573903,20200614,기타전문점,종로구,208,5054760
573904,20200614,기타전문점,중구,119,4728850


In [ ]:
# card_gu.to_csv('card_gu.csv', index = False)

## 유동인구와 카드데이터 결합

In [ ]:
pop_sum_final2.head()

,날짜,시군구,유동인구
0,20200104,강남구,4268654
1,20200104,강동구,2425402
2,20200104,강북구,1036862
3,20200104,강서구,2465692
4,20200104,관악구,1977717


In [ ]:
# 유동인구와 카드데이터 결합
df = card_gu.merge(pop_sum_final2, how = 'left')

In [ ]:
df.head()

,날짜,업종코드,업종명,시군구,판매건수,매출액,유동인구
0,20200104,1001,특급호텔,강남구,549,146315331,4268654
1,20200104,1001,특급호텔,중구,928,239811377,1669677
2,20200104,1002,1급호텔,중구,38,6941145,1669677
3,20200104,1020,기타숙박업,강남구,444,57200876,4268654
4,20200104,1020,기타숙박업,강동구,180,7928000,2425402


# 결합데이터 전처리

In [ ]:
df.head()

,base_ymd,mrhst_induty_cl_code,mrhst_induty_cl_nm,signgu_nm,selng_cascnt,salamt,popltn_cascnt2
0,2020-01-04,1001,특급호텔,강남구,549,146315331,4268654
1,2020-01-04,1001,특급호텔,중구,928,239811377,1669677
2,2020-01-04,1002,1급호텔,중구,38,6941145,1669677
3,2020-01-04,1020,기타숙박업,강남구,444,57200876,4268654
4,2020-01-04,1020,기타숙박업,강동구,180,7928000,2425402


In [ ]:
# 일자 Datetime으로 변환
df.base_ymd = pd.to_datetime(df.base_ymd, format = '%Y%m%d')

In [ ]:
# 각 열 이름 변환
df.columns = ['날짜', '업종코드', '업종명', '시군구', '판매건수', '매출액',  '유동인구합']

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 573906 entries, 0 to 573905
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   날짜      573906 non-null  datetime64[ns]
 1   업종코드    573906 non-null  int64         
 2   업종명     573906 non-null  object        
 3   시군구     573906 non-null  object        
 4   판매건수    573906 non-null  int64         
 5   매출액     573906 non-null  int64         
 6   유동인구합   573906 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 35.0+ MB


## 분석하지 않는 업종명 제거

In [ ]:
df.head()

,날짜,업종코드,업종명,시군구,판매건수,매출액,유동인구합
0,2020-01-04,1001,특급호텔,강남구,549,146315331,4268654
1,2020-01-04,1001,특급호텔,중구,928,239811377,1669677
2,2020-01-04,1002,1급호텔,중구,38,6941145,1669677
3,2020-01-04,1020,기타숙박업,강남구,444,57200876,4268654
4,2020-01-04,1020,기타숙박업,강동구,180,7928000,2425402


In [ ]:
# 제거할 업종명
delete = ['의료용품', '제약회사', '초중고교육기관', '기타운송', '화물운송', '사무통신기기수리', '사무기기', '중고자동차위탁판매', '자동차부품', 
          '위탁급식업', '미곡상', '옷감직물', '기타직물', '농축수산품', '기타건축자재', '기계공구', '비영리', '사무서비스', '기타대인서비스', '기타사무용품',
          '출판인쇄물', '기타유통업', '종합용역', '조세서비스(특약)', '조세서비스', '법률회계서비스(법인)', '법률회계서비스(개인)', '전자(상우회)',
          '기타용역서비스', '보일러펌프샷시', '과학기자재', '비씨카드정산용(할인)', '비영리/비대상', '보관창고업', '냉열기기', '기타광학품', '유리',
          '사무서비스(회원제형태)', '사업자미등록개인', '기타비영리유통', '중장비수리', '기타농업관련', '자체물품대금', '비료/농약/사료/종자', 
          '농기계', '건축요업품', '미용재료', '페인트', '윤활유전문판매', '기타연료', '유류판매']

In [ ]:
# 업종명 제거
df_del = df[~df['업종명'].isin(delete_str)].copy()
df_del

/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,날짜,업종코드,업종명,시군구,판매건수,매출액,유동인구합
0,2020-01-04,1001,특급호텔,강남구,549,146315331,4268654
1,2020-01-04,1001,특급호텔,중구,928,239811377,1669677
2,2020-01-04,1002,1급호텔,중구,38,6941145,1669677
3,2020-01-04,1020,기타숙박업,강남구,444,57200876,4268654
4,2020-01-04,1020,기타숙박업,강동구,180,7928000,2425402
...,...,...,...,...,...,...,...
573901,2020-06-14,9998,기타전문점,용산구,185,2793400,1848525
573902,2020-06-14,9998,기타전문점,은평구,42,1630800,1436435
573903,2020-06-14,9998,기타전문점,종로구,208,5054760,1269669
573904,2020-06-14,9998,기타전문점,중구,119,4728850,1166980


In [ ]:
# 제거 후 업종명 개수 확인
len(df_del.업종명.unique())

213

## 업종명 업계로 분류

In [ ]:
# 업종명들을 업계로 분류하기 위해 업종명 확인
df_del.업종명.unique()

array(['특급호텔', '1급호텔', '기타숙박업', '항공사', '관광여행', '택시', '골프용품', '스포츠레져용품',
       '악기점', '골프연습장', '스크린골프', '헬스크럽', '당구장', '노래방', '기타레져업', '화랑',
       '화방표구점', '민예공예품', '수족관', '화원', '애완동물', '영화관', '티켓', '일반가구', '기타가구',
       '가전제품', '기타전기제품', '주방용구', '주방용식기', '주유소', '카메라', '사진관',
       '자사카드발행백화점', '대형할인점', '편의점', '슈퍼마켓', '농축수산가공품', 'CATV홈쇼핑',
       '인터넷P/G', '인터넷종합Mall', '인터넷Mall', '상품권', '상품권전문판매', '전자상거래상품권전문판매',
       '구내매점(국가기관등)', '정장', '아동의류', '내의판매점', '스포츠의류', '맞춤복점', '기타의류',
       '침구수예점', '가방', '시계', '귀금속', '악세사리', '제화점', '신발', '기념품점', '기타잡화',
       '일반서적', '문구용품', '완구점', '기타서적문구', '외국어학원', '기능학원', '컴퓨터학원', '예체능학원',
       '보습학원', '유아원', '독서실', '기타교육', '컴퓨터', '통신기기', '전자(상우회)', '수입자동차',
       '이륜차판매', '자동차시트/타이어', '자동차정비', '세차장', '주차장', '종합병원', '병원', '의원',
       '한의원', '치과의원', '약국', '한약방', '동물병원', '이용원', '미용원', '피부미용실', '안경',
       '화장품', '사우나', '안마/스포츠마사지', '일반한식', '일식회집', '중국음식', '서양음식', '칵테일바',
       '주점', '스넥', '유흥주점', '단란주점', '제과점', '정육점', '주류판매점', '기타음료식품',
     

In [ ]:
# 업종명 분류 함수 정의
def sort_induty(x):
  a = ''
  if x in ['약국', '치과의원', '한의원', '의원', '병원', '종합병원', '기타의료기관및기타의료기기', '산후조리원', '한방병원', '조산원', '안경', '의료용품', 
           '치과병원', '건강진단', '제약회사']:
    a = '메디컬'
  elif x in ['기타교육', '보습학원', '예체능학원', '유아원', '학원(회원제형태)', '독서실', '외국어학원', '기능학원', '학습지교육', '초중고교육기관', 
             '유학원', '유치원', '컴퓨터학원', '교육테이프', '대학등록금']:
    a = '교육'
  elif x in ['미용원', '이용원']:
    a = '패션&뷰티'
  elif x in ['인터넷Mall', '인터넷종합Mall', '인터넷P/G', 'CATV홈쇼핑', '통신판매1', '통신판매2', '통신판매(생활서비스)', '통신판매(일반)']:
    a = '비대면쇼핑'
  elif x in ['고속버스', '택시회사', '택시', '철도', '항공사', '여객선', '렌트카', '기타운송', '기타교통수단', '화물운송']:
    a = '이동수단'
  elif x in ['CATV', '통신기기', '통신기기무이자할부', '카메라', '통신서비스', '통신서비스/비대상', '이동통신요금', '정보서비스', '컴퓨터', 
             '위성방송', '사무통신기기수리', '사무기기', '소프트웨어']:
    a = 'IT'
  elif x in ['중고자동차위탁판매', '중고자동차', '국산신차', '자동차부품', '자동차시트/타이어', '수입자동차', '국산신차직영부품/정비업소', 
             '국산신차직영부품/정비업소', '자동차서비스(회원제형태)', '카인테리어', '기타자동차서비스', '이륜차판매', '자동차정비']:
    a = '자동차'
  elif x in ['서양음식', '중국음식', '일식회집', '일반한식', '한정식', '기타건강식', '위탁급식업', '미곡상', '인삼제품', '건강식품(회원제형태)', '갈비전문점',
             '주점', '단란주점', '기타음료식품', '유흥주점', '칵테일바', '농축수산가공품', '홍삼제품']:
    a = 'F&B'
  elif x in ['1급호텔', '기타숙박업', '2급호텔', '특급호텔', '콘도']:
    a = '숙박'
  elif x in ['헬스크럽']:
    a = '피트니스'
  elif x in ['기타레져', '스포츠레져용품', '레져업소(회원제형태)', '스크린골프', '골프연습장', '골프용품', '문화취미기타', '테니스장', '볼링장', '수영장',
             '영화관', '스키장', '레져용품수리', '종합레져타운', '카지노', '수족관', '티켓', '화방표구점', '골프경기장', '기타레져업', '노래방', '당구장',
             '관광여행']:
    a = '문화&오락'
  elif x in ['부동산중개/임대', '부동산분양', '손해보험', '생명보험', '기타보험', 'P2P송금']:
    a = '금융&부동산'
  elif x in ['기타의류', '스포츠의류', '아동의류', '와이셔츠/타이', '맞춤복점', '정장', '옷감직물', '내의판매점', '기타직물', '양품점', '캐쥬얼의류', 
             '단체복']:
    a = '의류'
  elif x in ['농축수산품', '기타건축자재', '기계공구', '화원', '비영리', '사무서비스', '기타대인서비스', '기타사무용품', '출판인쇄물', '기타유통업',
             '종합용역', '조세서비스(특약)', '조세서비스', '기타전문점', '기타회원제형태업소4', '법률회계서비스(법인)', '회원제형태업소4', 
             '법률회계서비스(법인)', '혼례서비스', '법률회계서비스(개인)', '전자(상우회)', '기타용역서비스', '보일러펌프샷시', '과학기자재', '기념품점',
             '성인용품점', '비씨카드정산용(할인)', '기타수리서비스', '신변잡화수리', '비영리/비대상', '장례서비스', '기타회원제형태업소1', '기타회원제형태업소2',
             '기타회원제형태업소3', '보관창고업', '냉열기기', '기타광학품', '유리', '공공요금/대상', '사무서비스(회원제형태)', '견인서비스', '사업자미등록개인',
             '기타비영리유통', '중장비수리', '외국인전용가맹점', '기타회원제형태업소5', '골동품점', '혼수전문점', '기타농업관련', '자체물품대금', '총포류판매',
             '비료/농약/사료/종자', '공공요금', '농기계', '건축요업품', '미용재료', '페인트', '윤활유전문판매', '전자상거래상품권전문판매', '전자상거래상품권',
             '상품권전문판매', 'PG상품권', '기타연료', '유류판매']:
    a = '기타'
  else:
    a = '라이프스타일'
  return a

In [ ]:
# 업계 열 생성
df_del['업계'] = df.업종명.apply(sort_induty)

In [ ]:
df_del.head()

,날짜,업종코드,업종명,시군구,판매건수,매출액,유동인구합,업계
0,2020-01-04,1001,특급호텔,강남구,549,146315331,4268654,숙박
1,2020-01-04,1001,특급호텔,중구,928,239811377,1669677,숙박
2,2020-01-04,1002,1급호텔,중구,38,6941145,1669677,숙박
3,2020-01-04,1020,기타숙박업,강남구,444,57200876,4268654,숙박
4,2020-01-04,1020,기타숙박업,강동구,180,7928000,2425402,숙박


In [ ]:
# 업계 분류 확인
df_del.업계.unique()

array(['숙박', '이동수단', '문화&오락', '라이프스타일', '피트니스', '기타', 'IT', 'F&B',
       '비대면쇼핑', '의류', '교육', '자동차', '메디컬', '패션&뷰티', '금융&부동산'], dtype=object)

In [ ]:
df_del.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 474854 entries, 0 to 573905
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   날짜      474854 non-null  datetime64[ns]
 1   업종코드    474854 non-null  int64         
 2   업종명     474854 non-null  object        
 3   시군구     474854 non-null  object        
 4   판매건수    474854 non-null  int64         
 5   매출액     474854 non-null  int64         
 6   유동인구합   474854 non-null  int64         
 7   업계      474854 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 32.6+ MB


In [ ]:
df_del.head()

,날짜,업종코드,업종명,시군구,판매건수,매출액,유동인구합,업계
0,2020-01-04,1001,특급호텔,강남구,549,146315331,4268654,숙박
1,2020-01-04,1001,특급호텔,중구,928,239811377,1669677,숙박
2,2020-01-04,1002,1급호텔,중구,38,6941145,1669677,숙박
3,2020-01-04,1020,기타숙박업,강남구,444,57200876,4268654,숙박
4,2020-01-04,1020,기타숙박업,강동구,180,7928000,2425402,숙박


In [ ]:
# 업종코드 제거
df_del.drop('업종코드', axis = 1, inplace = True)

In [ ]:
df_del.head()

,날짜,업종명,시군구,판매건수,매출액,유동인구합,업계
0,2020-01-04,특급호텔,강남구,549,146315331,4268654,숙박
1,2020-01-04,특급호텔,중구,928,239811377,1669677,숙박
2,2020-01-04,1급호텔,중구,38,6941145,1669677,숙박
3,2020-01-04,기타숙박업,강남구,444,57200876,4268654,숙박
4,2020-01-04,기타숙박업,강동구,180,7928000,2425402,숙박


# 코로나 데이터 결합

In [ ]:
# 코로나 데이터
covid = pd.read_csv('monthly_covid19_seoul_long_0806.csv')

In [ ]:
covid.head()

,날짜,시군구,확진자수
0,2020-01-24,강남구,0
1,2020-01-24,강동구,0
2,2020-01-24,강북구,0
3,2020-01-24,강서구,1
4,2020-01-24,관악구,0


In [ ]:
covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2775 entries, 0 to 2774
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   날짜      2775 non-null   datetime64[ns]
 1   시군구     2775 non-null   object        
 2   확진자수    2775 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 65.2+ KB


In [ ]:
# 날짜를 datetime으로 변환
covid.날짜 = pd.to_datetime(covid.날짜)

In [ ]:
covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2775 entries, 0 to 2774
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   날짜      2775 non-null   datetime64[ns]
 1   시군구     2775 non-null   object        
 2   확진자수    2775 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 65.2+ KB


# 카드 & 유동인구 데이터와 코로나 데이터 결합

In [ ]:
# 카드유동인구 데이터와 코로나 데이터 결합
card_covid = df_del.merge(covid, how = 'left')

In [ ]:
card_covid.head()

,날짜,업종명,시군구,판매건수,매출액,유동인구합,업계,확진자수
0,2020-01-04,특급호텔,강남구,549,146315331,4268654,숙박,NaN
1,2020-01-04,특급호텔,중구,928,239811377,1669677,숙박,NaN
2,2020-01-04,1급호텔,중구,38,6941145,1669677,숙박,NaN
3,2020-01-04,기타숙박업,강남구,444,57200876,4268654,숙박,NaN
4,2020-01-04,기타숙박업,강동구,180,7928000,2425402,숙박,NaN


In [ ]:
card_covid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 474854 entries, 0 to 474853
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   날짜      474854 non-null  datetime64[ns]
 1   업종명     474854 non-null  object        
 2   시군구     474854 non-null  object        
 3   판매건수    474854 non-null  int64         
 4   매출액     474854 non-null  int64         
 5   유동인구합   474854 non-null  int64         
 6   업계      474854 non-null  object        
 7   확진자수    369292 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(3), object(3)
memory usage: 52.6+ MB


In [ ]:
# 확진자수가 결측치인 값들 채우고 정수로 변환
card_covid.확진자수 = card_covid.확진자수.fillna(0).astype(int)

In [ ]:
card_covid

,날짜,업종명,시군구,판매건수,매출액,유동인구합,업계,확진자수
0,2020-01-04,특급호텔,강남구,549,146315331,4268654,숙박,0
1,2020-01-04,특급호텔,중구,928,239811377,1669677,숙박,0
2,2020-01-04,1급호텔,중구,38,6941145,1669677,숙박,0
3,2020-01-04,기타숙박업,강남구,444,57200876,4268654,숙박,0
4,2020-01-04,기타숙박업,강동구,180,7928000,2425402,숙박,0
...,...,...,...,...,...,...,...,...
474849,2020-06-14,기타전문점,용산구,185,2793400,1848525,기타,1
474850,2020-06-14,기타전문점,은평구,42,1630800,1436435,기타,1
474851,2020-06-14,기타전문점,종로구,208,5054760,1269669,기타,0
474852,2020-06-14,기타전문점,중구,119,4728850,1166980,기타,0


In [ ]:
# 확진자수/유동인구 계산 예시
df1 = card_covid.groupby(['날짜', '시군구'])['확진자수'].first().reset_index()
df1[(df1.날짜 > '2020-01-24') & (df1.날짜 < '2020-02-01')]

,날짜,시군구,확진자수
525,2020-01-25,강남구,0
526,2020-01-25,강동구,0
527,2020-01-25,강북구,0
528,2020-01-25,강서구,0
529,2020-01-25,관악구,0
...,...,...,...
695,2020-01-31,용산구,0
696,2020-01-31,은평구,0
697,2020-01-31,종로구,2
698,2020-01-31,중구,0


In [ ]:
# 파일 저장
card_covid.to_csv('인구카드코로나.csv', index = False)